# FNC 6.8 Zero-stability
For each multistep method (a)–(f), we check:

1. **Consistency (order ≥ 1)**:  
   $$\sum_j \alpha_j = 0, \qquad \sum_j j\,\alpha_j = \sum_j \beta_j.$$

2. **Zero-stability** via the **root condition** for \(\rho(z)=\sum_j \alpha_j z^j\):  
   all roots satisfy \(|r|\le 1\), and any root with \(|r|=1\) is **simple**.


In [4]:

import numpy as np

def consistency_report(alpha, beta, tol=1e-12):
    """Check Σα=0 and Σ j α_j = Σ β_j."""
    s0 = sum(alpha.values())
    s1 = sum(j*a for j,a in alpha.items())
    b0 = sum(beta.values())
    ok1 = abs(s0) < tol
    ok2 = abs(s1 - b0) < tol
    return ok1, ok2, (s0, s1, b0)

def rho_poly(alpha):
    """Return numpy.poly1d for ρ(z)=Σ α_j z^j with nonnegative powers."""
    if not alpha:
        return np.poly1d([0.0])
    shifts = list(alpha.keys())
    shift_up = -min(shifts) if min(shifts) < 0 else 0
    deg = max(shifts) + shift_up
    coeffs = np.zeros(deg+1, dtype=float)
    for j,a in alpha.items():
        # highest degree first
        coeffs[deg - (j + shift_up)] += a
    return np.poly1d(coeffs)

def root_condition(alpha, tol=1e-8):
    p = rho_poly(alpha)
    roots = np.roots(p)
    absr = np.abs(roots)
    # max modulus <= 1 (allow tiny numeric slack)
    bounded = (absr.max() <= 1.0 + 1e-10) if roots.size else True
    # check simplicity on the unit circle
    simple = True
    for i in range(len(roots)):
        if abs(absr[i] - 1.0) < 1e-6:
            # multiplicity: count near duplicates
            mult = np.sum(np.abs(roots - roots[i]) < 1e-8)
            if mult > 1:
                simple = False
                break
    return roots, (bounded and simple)


## Methods (a)–(f) as `(alpha, beta)` with integer time shifts

In [5]:

# (a) BD2: (3/2)u_{i+1} - 2u_i + (1/2)u_{i-1} = h f_{i+1}
alpha_a = {1: 1.5, 0: -2.0, -1: 0.5};   beta_a = {1: 1.0}

# (b) BD3: (11/6)u_{i+1} - 3u_i + (3/2)u_{i-1} - (1/3)u_{i-2} = h f_{i+1}
alpha_b = {1: 11/6, 0: -3.0, -1: 1.5, -2: -1/3};   beta_b = {1: 1.0}

# (c) u_{i+1} = u_{i-1} + 2 h f_i
alpha_c = {1: 1.0, -1: -1.0};           beta_c = {0: 2.0}

# (d) u_{i+1} = -u_i + u_{i-1} + u_{i-2} + (2h/3)(4 f_i + f_{i-1} + f_{i-2})
# => u_{i+1} + u_i - u_{i-1} - u_{i-2} - h[(8/3)f_i + (2/3)f_{i-1} + (2/3)f_{i-2}] = 0
alpha_d = {1: 1.0, 0: 1.0, -1: -1.0, -2: -1.0};    beta_d = {0: 8/3, -1: 2/3, -2: 2/3}

# (e) u_{i+1} = u_{i-3} + (4h/3)(2 f_i - f_{i-1} + 2 f_{i-2})
# => u_{i+1} - u_{i-3} - h[(8/3)f_i - (4/3)f_{i-1} + (8/3)f_{i-2}] = 0
alpha_e = {1: 1.0, -3: -1.0};           beta_e = {0: 8/3, -1: -4/3, -2: 8/3}

# (f) u_{i+1} = -2u_i + 3u_{i-1} + h ( f_{i+1} + 2 f_i + f_{i-1} )
# => u_{i+1} + 2u_i - 3u_{i-1} - h( f_{i+1} + 2 f_i + f_{i-1} ) = 0
alpha_f = {1: 1.0, 0: 2.0, -1: -3.0};   beta_f = {1: 1.0, 0: 2.0, -1: 1.0}

methods = {
    "(a) BD2": (alpha_a, beta_a),
    "(b) BD3": (alpha_b, beta_b),
    "(c)":     (alpha_c, beta_c),
    "(d)":     (alpha_d, beta_d),
    "(e)":     (alpha_e, beta_e),
    "(f)":     (alpha_f, beta_f),
}


## Reports: consistency and zero-stability

In [6]:

np.set_printoptions(precision=6, suppress=True)
for name, (alpha, beta) in methods.items():
    ok1, ok2, sums = consistency_report(alpha, beta)
    roots, zs = root_condition(alpha)
    print(f"{name}:")
    print(f"  Consistency  Σα=0? {ok1}   Σ jα = Σβ? {ok2}   (sums={sums})")
    print(f"  rho(z) roots: {roots}")
    print(f"  Zero-stable? {zs}")


(a) BD2:
  Consistency  Σα=0? True   Σ jα = Σβ? True   (sums=(0.0, 1.0, 1.0))
  rho(z) roots: [1.       0.333333]
  Zero-stable? True
(b) BD3:
  Consistency  Σα=0? True   Σ jα = Σβ? True   (sums=(-5.551115123125783e-17, 0.9999999999999999, 1.0))
  rho(z) roots: [1.      +0.j       0.318182+0.283864j 0.318182-0.283864j]
  Zero-stable? True
(c):
  Consistency  Σα=0? True   Σ jα = Σβ? True   (sums=(0.0, 2.0, 2.0))
  rho(z) roots: [-1.  1.]
  Zero-stable? True
(d):
  Consistency  Σα=0? True   Σ jα = Σβ? True   (sums=(0.0, 4.0, 3.9999999999999996))
  rho(z) roots: [ 1. -1. -1.]
  Zero-stable? False
(e):
  Consistency  Σα=0? True   Σ jα = Σβ? True   (sums=(0.0, 4.0, 4.0))
  rho(z) roots: [-1.+0.j  0.+1.j  0.-1.j  1.+0.j]
  Zero-stable? True
(f):
  Consistency  Σα=0? True   Σ jα = Σβ? True   (sums=(0.0, 4.0, 4.0))
  rho(z) roots: [-3.  1.]
  Zero-stable? False
